In [2]:
import pandas as pd
import numpy as np

import re
import string

import spacy

import gensim
from gensim import corpora

# libraries for visualization
import pyLDAvis
import pyLDAvis.gensim
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [4]:
review_data= pd.read_csv("Reviews.csv")
print(review_data.head(2))
print(len(review_data))
print('Unique Products')
print(len(review_data.groupby('ProductId')))
print('Unique Users')
print(len(review_data.groupby('UserId')))

   Id   ProductId          UserId ProfileName  HelpfulnessNumerator  \
0   1  B001E4KFG0  A3SGXH7AUHU8GW  delmartian                     1   
1   2  B00813GRG4  A1D87F6ZCVE5NK      dll pa                     0   

   HelpfulnessDenominator  Score        Time                Summary  \
0                       1      5  1303862400  Good Quality Dog Food   
1                       0      1  1346976000      Not as Advertised   

                                                Text  
0  I have bought several of the Vitality canned d...  
1  Product arrived labeled as Jumbo Salted Peanut...  
568454
Unique Products
74258
Unique Users
256059


In [5]:
def clean_text(text ): 
    delete_dict = {sp_character: '' for sp_character in string.punctuation} 
    delete_dict[' '] = ' ' 
    table = str.maketrans(delete_dict)
    text1 = text.translate(table)
    #print('cleaned:'+text1)
    textArr= text1.split()
    text2 = ' '.join([w for w in textArr if ( not w.isdigit() and  ( not w.isdigit() and len(w)>3))]) 
    
    return text2.lower()

In [21]:
import nltk
nltk.download('stopwords') # run this one time

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [7]:
review_data.dropna(axis = 0, how ='any',inplace=True) 

review_data['Text'] = review_data['Text'].apply(clean_text)
review_data['Num_words_text'] = review_data['Text'].apply(lambda x:len(str(x).split())) 

print('-------Dataset --------')
print(review_data['Score'].value_counts())
print(len(review_data))
print('-------------------------')
max_review_data_sentence_length  = review_data['Num_words_text'].max()

mask = (review_data['Num_words_text'] < 100) & (review_data['Num_words_text'] >=20)
df_short_reviews = review_data[mask]
df_sampled = df_short_reviews.groupby('Score').apply(lambda x: x.sample(n=20000)).reset_index(drop = True)

print('No of Short reviews')
print(len(df_short_reviews))

-------Dataset --------
5    363111
4     80655
1     52264
3     42638
2     29743
Name: Score, dtype: int64
568411
-------------------------
No of Short reviews
373281


In [8]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
# function to remove stopwords
def remove_stopwords(text):
    textArr = text.split(' ')
    rem_text = " ".join([i for i in textArr if i not in stop_words])
    return rem_text

# remove stopwords from the text
df_sampled['Text']=df_sampled['Text'].apply(remove_stopwords)

In [12]:
nlp = spacy.load('en_core_web_md', disable=['parser', 'ner'])

def lemmatization(texts,allowed_postags=['NOUN', 'ADJ']): 
    output = []
    for sent in texts:
        doc = nlp(sent) 
        output.append([token.lemma_ for token in doc if token.pos_ in allowed_postags ])
    return output

In [28]:
text_list=df_sampled['Text'].tolist()
print(text_list[1])
tokenized_reviews = lemmatization(text_list)
print(tokenized_reviews[1])

drinking year last boxes different much weaker kcups coffee still weak favorite anymore find favorite


KeyboardInterrupt: 

In [34]:
len(dictionary)

45428

In [15]:
dictionary = corpora.Dictionary(tokenized_reviews)
doc_term_matrix = [dictionary.doc2bow(rev) for rev in tokenized_reviews]

In [16]:
LDA = gensim.models.ldamodel.LdaModel

# Build LDA model
lda_model = LDA(corpus=doc_term_matrix, id2word=dictionary, num_topics=10, random_state=100,
                chunksize=1000, passes=50,iterations=100)

In [17]:
lda_model.print_topics()

[(0,
  '0.033*"time" + 0.019*"great" + 0.015*"small" + 0.015*"minute" + 0.015*"bottle" + 0.015*"little" + 0.014*"work" + 0.012*"pasta" + 0.011*"size" + 0.011*"large"'),
 (1,
  '0.064*"flavor" + 0.034*"good" + 0.032*"taste" + 0.015*"great" + 0.014*"love" + 0.014*"green" + 0.012*"nice" + 0.012*"favorite" + 0.011*"brand" + 0.011*"drink"'),
 (2,
  '0.074*"chip" + 0.043*"butter" + 0.041*"peanut" + 0.030*"blend" + 0.024*"popcorn" + 0.023*"potato" + 0.020*"yummy" + 0.017*"bag" + 0.017*"snack" + 0.016*"flavorful"'),
 (3,
  '0.041*"salt" + 0.035*"flavor" + 0.024*"protein" + 0.022*"spicy" + 0.016*"diet" + 0.015*"pepper" + 0.015*"heat" + 0.014*"taste" + 0.013*"syrup" + 0.012*"great"'),
 (4,
  '0.045*"product" + 0.044*"price" + 0.043*"store" + 0.039*"amazon" + 0.029*"great" + 0.029*"good" + 0.021*"order" + 0.020*"time" + 0.015*"year" + 0.014*"grocery"'),
 (5,
  '0.062*"sauce" + 0.036*"cheese" + 0.032*"soup" + 0.020*"noodle" + 0.016*"cook" + 0.015*"local" + 0.015*"tomato" + 0.015*"dish" + 0.014*"co

In [18]:
# Visualize the topics
#https://github.com/bmabey/pyLDAvis
#https://speakerdeck.com/bmabey/visualizing-topic-models
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, doc_term_matrix, dictionary)
vis

C:\Users\User\anaconda3\envs\newtensorflow\lib\site-packages\pyLDAvis\_prepare.py:243: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.114133 -0.089422       1        1  16.613831
4     -0.178076  0.154759       2        1  16.040416
9     -0.124866  0.084790       3        1  15.597185
6     -0.076881 -0.046007       4        1  11.083436
0     -0.069788  0.187922       5        1  10.135822
8     -0.074469 -0.047118       6        1   8.277793
7      0.096319 -0.251816       7        1   6.881308
3      0.051990 -0.198989       8        1   5.825599
2      0.075221  0.049388       9        1   5.445477
5      0.414682  0.156492      10        1   4.099131, topic_info=          Term          Freq         Total Category  logprob  loglift
19      coffee  30798.000000  30798.000000  Default  30.0000  30.0000
44        food  22329.000000  22329.000000  Default  29.0000  29.0000
160     flavor  32586.000000  32586.000000  Default  28.0000  28.0000
79       store  13896.000000  13896.000000  Default  27.0000  27.0000
200      sugar   8893.000000   8893.000000  Default  26.0000  26.0000
...        ...           ...           ...      ...      ...      ...
440      local   1231.086829   5938.033314  Topic10  -4.1990   1.6209
1477  homemade    631.240872   1010.813617  Topic10  -4.8669   2.7236
709     recipe    869.186576   2671.457027  Topic10  -4.5471   2.0716
622      fresh    713.241995   6056.652459  Topic10  -4.7448   1.0553
705     family    596.123257   4009.837819  Topic10  -4.9242   1.2883

[531 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
2863      9  0.999216  addictive
5776      7  0.998600      adult
673       5  0.998844  afternoon
2502      1  0.999601    allergy
386       7  0.999476     almond
...     ...       ...        ...
26        4  0.070735       year
26        6  0.041347       year
6308     10  0.998289     yogurt
1647      9  0.998245      young
1830      9  0.999751      yummy

[768 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 5, 10, 7, 1, 9, 8, 4, 3, 6])

In [19]:
print('\nPerplexity: ', lda_model.log_perplexity(doc_term_matrix,total_docs=10000))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
from gensim.models.coherencemodel import CoherenceModel
coherence_model_lda = CoherenceModel(model=lda_model, texts=tokenized_reviews, dictionary=dictionary , coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -8.844296513043854

Coherence Score:  0.4332053140362627


In [20]:
def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    texts : List of input texts
    limit : Max num of topics

    Returns:
    -------
    model_list : List of LDA topic models
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model = gensim.models.ldamodel.LdaModel(corpus=corpus, num_topics=num_topics, id2word=dictionary)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values

In [ ]:
model_list, coherence_values = compute_coherence_values(dictionary=dictionary, corpus=doc_term_matrix, texts=tokenized_reviews, start=2, limit=50, step=1)

In [ ]:
# Show graph
limit=50; start=2; step=1;
x = range(start, limit, step)
plt.plot(x, coherence_values)
plt.xlabel("Num Topics")
plt.ylabel("Coherence score")
plt.legend(("coherence_values"), loc='best')
plt.show()# Print the coherence scores

In [ ]:
# Print the coherence scores
for m, cv in zip(x, coherence_values):
    print("Num Topics =", m, " has Coherence Value of", round(cv, 4))

In [ ]:
# Select the model and print the topics
optimal_model = model_list[7]
model_topics = optimal_model.show_topics(formatted=False)
optimal_model.print_topics(num_words=10)

In [ ]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(optimal_model, doc_term_matrix, dictionary)
vis